# Colour Correction using Splines

### Define constants

In [47]:
# Enable automatic reloading of modules before executing code to ensure latest changes are used.
%load_ext autoreload
%autoreload 2

# Importing necessary libraries and functions.
# 'colour' is used for color science computations.
# 'data' contains functions to load various datasets and color matching functions.
import colour
from data import (load_dataset_sfu, load_dataset_csv, load_cmfs, 
                  load_camera, load_insitu, msds_to_rgb, msds_to_xyz, load_dataset_skin)
import numpy as np

# Setting a seed for random number generation to ensure reproducibility.
RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)

# Defining file paths for the datasets. These files should contain the data needed for analysis.
SFU_FILE_PATH = 'data/reflect_db.reflect'  # SFU material reflectance database
CAVE_FOSTER2004_PATH = 'data/cave_foster2002.csv'  # CAVE dataset for color constancy
INSITU_PATH = "data/insitu_dataset.csv"  # In-situ measurements dataset
CAVE_PATH = 'data/cave.csv'  # Another CAVE dataset file
FOSTER_50_PATH = 'data/foster50.csv'  # Foster dataset with 50 images for color analysis
CAMERA = 'sigma'  # Specifying the camera model used in the datasets or for simulation


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
cave_foster2004_dataset = load_dataset_csv(CAVE_FOSTER2004_PATH)
foster_50_dataset = load_dataset_csv(FOSTER_50_PATH)
insitu = load_insitu(INSITU_PATH)


In [45]:
TRAIN = cave_foster2004_dataset
TEST = foster_50_dataset
VALIDATION = cave_foster2004_dataset

### Computing Observer Responses
We can easily change the order of test and train sets here

In [54]:
from plotting import plot_chromaticity_diagram
from colour.characterisation import training_data_sds_to_XYZ

cmfs, illuminant = load_cmfs()
response_trainset_xyz = msds_to_xyz(TRAIN, cmfs, illuminant)
response_testset_xyz = msds_to_xyz(TEST, cmfs, illuminant)

### Computing Camera Responses

In [55]:
from colour.characterisation import normalise_illuminant, training_data_sds_to_RGB
import numpy as np
MSDS_TRAIN = load_camera(CAMERA)

response_trainset_camera = msds_to_rgb(TRAIN,MSDS_TRAIN, illuminant)
response_testset_camera = msds_to_rgb(TEST,MSDS_TRAIN, illuminant)

### Fit Generalized Additive Model with P-splines

### Nikon

In [ ]:
from models import GAMOptimizer
from evaluate import pred
def test_gam_optimizer(splines_lams_dict, order_value=3):
    """
    Test GAMOptimizer models with different numbers of splines and corresponding lambda values.

    Parameters:
    - splines_lams_dict: Dictionary where keys are spline numbers and values are the corresponding lambda values for regularization.
    - order_value: The order of the spline. Default is 3.
    """
    for n_splines, lams_value in splines_lams_dict.items():
        print(f"Testing GAMOptimizer with {n_splines} splines and lambda {lams_value}")
        gam = GAMOptimizer(lams=lams_value, order=order_value, n_splines=n_splines)
        gam.fit(response_trainset_camera, response_trainset_xyz)
        pred(gam, response_testset_camera, response_testset_xyz, f"DeltaE Foster+CAVE with {n_splines} splines and lambda {lams_value}")

# Example usage: Create a dictionary mapping spline numbers to their corresponding lambda values and pass it to the function.
splines_lams_dict = {
    5: 1e-6,
    10: 0.0001,
    20: 0.01
}
if CAMERA == 'nikon':
    test_gam_optimizer(splines_lams_dict)

### Sigma

In [ ]:
def test_gam_optimizer(splines_lams_dict, order_value=3):
    """
    Test GAMOptimizer models with different numbers of splines and corresponding lambda values.

    Parameters:
    - splines_lams_dict: Dictionary where keys are spline numbers and values are the corresponding lambda values for regularization.
    - order_value: The order of the spline. Default is 3.
    """
    for n_splines, lams_value in splines_lams_dict.items():
        print(f"Testing GAMOptimizer with {n_splines} splines and lambda {lams_value}")
        gam = GAMOptimizer(lams=lams_value, order=order_value, n_splines=n_splines)
        gam.fit(response_trainset_camera, response_trainset_xyz)
        pred(gam, response_testset_camera, response_testset_xyz, f"{n_splines} splines and lambda {lams_value}")

# Example usage: Create a dictionary mapping spline numbers to their corresponding lambda values and pass it to the function.
splines_lams_dict = {
    5: 1e-9,
    10: 0.0001,
    20: 0.0001
}
if CAMERA == 'sigma':
    test_gam_optimizer(splines_lams_dict)

In [58]:
from models import RGBtoXYZNetwork
from evaluate import pred
mcdonalds_nn = RGBtoXYZNetwork()
mcdonalds_nn.fit(response_trainset_camera, response_trainset_xyz)
pred(mcdonalds_nn, response_testset_camera, response_testset_xyz, "Foster 50")

Epoch 1, Train Loss: 30.058632578168595, Validation Loss: 29.240988731384277
Epoch 2, Train Loss: 26.26083483014788, Validation Loss: 26.54704189300537
Epoch 3, Train Loss: 24.2638156073434, Validation Loss: 24.954862117767334
Epoch 4, Train Loss: 22.186376776014054, Validation Loss: 24.29143524169922
Epoch 5, Train Loss: 21.836743354797363, Validation Loss: 24.049461364746094
Epoch 6, Train Loss: 21.684913771493093, Validation Loss: 24.14634418487549
Epoch 7, Train Loss: 21.680494035993302, Validation Loss: 23.839200019836426
Epoch 8, Train Loss: 21.302258150918142, Validation Loss: 25.334771633148193
Epoch 9, Train Loss: 21.611083848135813, Validation Loss: 23.934657096862793
Epoch 10, Train Loss: 21.114169597625732, Validation Loss: 23.711734294891357
Epoch 11, Train Loss: 21.28043236051287, Validation Loss: 23.77201747894287
Epoch 12, Train Loss: 21.565985815865652, Validation Loss: 23.417031288146973
Epoch 13, Train Loss: 21.031372138432094, Validation Loss: 24.191319942474365
Epo

### Fit Linear Model

In [ ]:
from sklearn.linear_model import LinearRegression

linear = LinearRegression(fit_intercept=False)

print(np.max(response_trainset_camera))


linear.fit(response_trainset_camera, response_trainset_xyz)
pred(linear, response_testset_camera, response_testset_xyz, "Foster 50")


In [ ]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=1))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

### Fit 3rd order Root-Polynomial Model

In [ ]:
from models import PolynomialTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])

RP_linear_3.fit(response_trainset_camera, response_trainset_xyz)

pred(RP_linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


### Fit a 2nd order Root-Polynomial Model

In [ ]:
from models import GAMOptimizer, PolynomialTransformer, DeltaEOptimizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_2 = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])


RP_linear_2.fit(response_trainset_camera, response_trainset_xyz)
pred(RP_linear_2, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


In [ ]:
P_Linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_Linear_3.fit(response_trainset_camera, response_trainset_xyz)
pred(P_Linear_3, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

In [ ]:
P_linear = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_linear.fit(response_trainset_camera, response_trainset_xyz)
pred(P_linear, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")


In [ ]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000P = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=3))
])


DE2000P.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000P, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")

In [ ]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=True, degree=3))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred(DE2000RP, response_testset_camera, response_testset_xyz, "DeltaE Foster+CAVE")